In [3]:
# -*- coding: utf-8 -*-
"""
Created on Mon Sep 12 06:37:07 2022
Adaptación de CirSeq a reads de 150 nt
@author: Rocío Marugán (rmaruganpin@alumni.unav.es)
"""

#%% Obtención de la unidad repetitiva
from scipy.stats import mode
import gzip

infile = gzip.open("T2314_WT_0h_S5_R1_001_v2.fastq.gz","r") 
outfile1 = gzip.open("T2314_WT_0h_S5_R1_001_consensus.fastq.gz","wb")
outfile2 = open("T2314_WT_0h_S5_R1_001_consensus.txt","w")
# outfile3 = gzip.open("T2314_WT_0h_S5_R1_001_no_repetition.fastq.gz", "wb")
# outfile5 = gzip.open("T2314_WT_0h_S5_R1_001_consensus_under.fastq.gz","wb")

RepeatLengths = list()
j = 0
# no_repetition = 0
while True:
    
    SequenceID = infile.readline().decode().rstrip("\n")

    if SequenceID == "":
        break

    Sequence = infile.readline().decode()
    Sequence = Sequence.rstrip("\n") 
    EmptyLine = infile.readline()
    QualityScores = infile.readline().decode()
    QualityScores = QualityScores.rstrip("\n")
    
    SubstringLengths = []
    #Identificación de todos los posibles sets de repeticiones y guardado de su longitud

    i = 1
    while i <= 142: # Lecturas de 150 nt
        Substrings = Sequence.split(Sequence[i:i+8])
        for String in Substrings[1:-1]: #[1:-1] evita los extremos, que probablemente no coincidan con la longitud exacta de repetición
            SubstringLengths.append(len(String))
        i += 1
    
    if bool(len(SubstringLengths)):
        SubstringLengthsMode = mode(SubstringLengths) 
        RepeatLength = int(SubstringLengthsMode[0][0]) + 8 
        RepeatLengths.append(RepeatLength)
        ConsensusSequence = Sequence[:RepeatLength]
        
        # Check-point: buscamos un mínimo de unidad repetitiva de 20 nt
        if ConsensusSequence[0:19] == Sequence[len(ConsensusSequence):len(ConsensusSequence)+20]:
            SequenceID = SequenceID + ":" + str(RepeatLength) + "\n"
            ConsensusSequence = ConsensusSequence + "\n"
            QualityScores = QualityScores[:RepeatLength] + "\n"
        
            if RepeatLength >= 20:
                outfile1.write(SequenceID.encode()) 
                outfile1.write(ConsensusSequence.encode()) 
                outfile1.write(EmptyLine) 
                outfile1.write(QualityScores.encode()) 
                print(ConsensusSequence)
            
            # else:
            #     outfile5.write(SequenceID.encode()) 
            #     outfile5.write(ConsensusSequence.encode()) 
            #     outfile5.write(EmptyLine) 
            #     outfile5.write(QualityScores.encode())
    #     else:
    #         no_repetition += 1
            
    #         SequenceID = SequenceID + "\n"
    #         Sequence = Sequence + "\n"
    #         QualityScores = QualityScores + "\n"
    #         outfile3.write(SequenceID.encode())
    #         outfile3.write(Sequence.encode())
    #         outfile3.write(EmptyLine)
    #         outfile3.write(QualityScores.encode())
    
    # else:
    #     no_repetition += 1
        
    #     SequenceID = SequenceID + "\n"
    #     Sequence = Sequence + "\n"
    #     QualityScores = QualityScores + "\n"
    #     outfile3.write(SequenceID.encode())
    #     outfile3.write(Sequence.encode())
    #     outfile3.write(EmptyLine)
    #     outfile3.write(QualityScores.encode())
        
    j = j+1

outfile2.write(str(RepeatLengths))

infile.close()
outfile1.close()
outfile2.close()
# outfile3.close()
# outfile5.close()



In [ ]:
#%% Obtención de todas las permutaciones de la unidad repetitiva

def rotate(Sequence):
    i = 0
    
    while i < len(Sequence):
        SequenceID_2 = "@" + "rotation" + str(i) + SequenceID
        outfile4.write(SequenceID_2.encode())
        Sequence_2 = Sequence[-i:] + Sequence[:-i] + "\n"
        outfile4.write(Sequence_2.encode())
        outfile4.write(EmptyLine)
        QualityScores_2 = QualityScores + "\n"
        outfile4.write(QualityScores_2.encode())
        i += 1
        
infile = gzip.open("DS_0h_S4_R1_001_consensus_over.fastq.gz", "r")
outfile4 = gzip.open("DS_0h_S4_R1_001_rotated.fastq.gz","wb")

while True:
    
    SequenceID = infile.readline().decode().replace("@","")    

    if SequenceID == "":
        break
 
    Sequence = infile.readline().decode()
    Sequence = Sequence.rstrip("\n") 
    EmptyLine = infile.readline()
    QualityScores = infile.readline().decode()
    QualityScores = QualityScores.rstrip("\n")
    
    Sequence = rotate(Sequence)
    
infile.close()
outfile4.close()